In [17]:
import pandas as pd
import requests
import time
from datetime import date,datetime

pd.set_option('display.max_columns', None)
import os

from utils import *


In [18]:
from dotenv import load_dotenv
load_dotenv()


True

In [19]:
token = os.getenv('data_access_token')
client_id = int(os.getenv('client_id'))
business_timezone_offset = int(os.getenv('business_timezone_offset'))



# Branches

In [20]:
last_page = call_foodics('branches', 1, client_id, token, return_last_page=True)

list_responses = call_foodics('branches', last_page, client_id, token)

page 1
branches?page=1
page 1
branches?page=1


In [21]:
df_branches = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_branches['client_id'] = client_id
df_branches['slug'] = df_branches.name.apply(lambda x: generate_slug(x))
df_branches

,id,name,name_localized,reference,type,latitude,longitude,phone,opening_from,opening_to,inventory_end_of_day_time,receipt_header,receipt_footer,settings,created_at,updated_at,deleted_at,receives_online_orders,accepts_reservations,reservation_duration,reservation_times,address,client_id,slug
0,964fde3e-a16d-4136-b433-f81d2c1d9b12,Abdali Boulevard,None,B01,1,0,0,None,04:00,04:00,04:00,None,كيف قهوتك؟,"{'branch_tax_number': '178171182', 'sa_zatca_b...",2022-05-16 10:42:32,2023-01-16 09:15:16,None,True,False,30,None,None,2,abdali-boulevard
1,964fde3e-a371-426e-a209-d6f3ff79d2a1,Sports City,None,B02,1,0,0,None,04:00,04:00,04:00,None,كيف قهوتك؟,"{'branch_tax_number': '178171182', 'sa_zatca_b...",2022-05-16 10:42:32,2023-01-16 09:16:20,None,True,False,30,None,Sport City,2,sports-city
2,964fde3e-a88f-4426-b476-62d018d33feb,Abdoun,None,B03,1,0,0,None,04:00,04:00,04:00,None,كيف قهوتك؟,"{'branch_tax_number': '178171182', 'sa_zatca_b...",2022-05-16 10:42:32,2023-01-16 09:17:01,None,True,False,30,None,None,2,abdoun
3,964fde3e-aa4d-48aa-8c29-e0c8c77cf0b6,Fuhais,None,B04,1,0,0,None,04:00,04:00,04:00,None,كيف قهوتك؟,"{'branch_tax_number': '178171182', 'sa_zatca_b...",2022-05-16 10:42:32,2023-01-16 09:17:23,None,True,False,30,None,None,2,fuhais
4,964fde3e-abf8-476c-b139-cc314022c24f,7th Circle,None,B05,1,0,0,None,04:00,04:00,04:00,None,كيف قهوتك؟,"{'branch_tax_number': '178171182', 'sa_zatca_b...",2022-05-16 10:42:32,2023-05-02 11:05:39,None,True,False,30,None,None,2,7th-circle
5,964fde3e-aec3-41f6-8fc9-b201a919e1dd,Events Backup (Previously Main Warehouse),None,B06,1,None,None,None,04:00,04:00,04:00,None,كيف قهوتك؟,"{'branch_tax_number': '178171182', 'sa_zatca_b...",2022-05-16 10:42:32,2023-03-22 13:36:05,None,True,False,30,None,None,2,events-backup-previously-main-warehouse
6,96ed24b8-517e-465f-b29c-ea84ec013766,Kia Showroom,None,B07,1,None,None,None,04:00,04:00,04:00,None,None,"{'sa_zatca_branch_address': [], 'display_backg...",2022-08-02 15:04:34,2023-01-16 09:14:49,None,True,False,30,None,None,2,kia-showroom
7,96fecd4d-69bd-4669-aeef-a4724c147e9b,Event Branch,None,B08,1,None,None,None,04:00,04:00,04:00,None,None,"{'sa_zatca_branch_address': [], 'display_backg...",2022-08-11 09:45:02,2023-01-16 09:08:03,None,True,False,30,None,None,2,event-branch
8,97c3bc91-eba9-4732-be73-ca9c80081518,Zarqa - Karama,None,B09,1,None,None,None,04:00,04:00,04:00,None,None,"{'sa_zatca_branch_address': [], 'display_backg...",2022-11-17 07:16:20,2023-01-16 09:13:56,None,True,False,30,None,None,2,zarqa-karama


In [22]:
df_branches = df_branches[['id', 'client_id', 'name', 'slug', 'opening_from', 'opening_to', 'created_at', 'updated_at', 'deleted_at']]

In [23]:
df_branches

,id,client_id,name,slug,opening_from,opening_to,created_at,updated_at,deleted_at
0,964fde3e-a16d-4136-b433-f81d2c1d9b12,2,Abdali Boulevard,abdali-boulevard,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:15:16,None
1,964fde3e-a371-426e-a209-d6f3ff79d2a1,2,Sports City,sports-city,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:16:20,None
2,964fde3e-a88f-4426-b476-62d018d33feb,2,Abdoun,abdoun,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:17:01,None
3,964fde3e-aa4d-48aa-8c29-e0c8c77cf0b6,2,Fuhais,fuhais,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:17:23,None
4,964fde3e-abf8-476c-b139-cc314022c24f,2,7th Circle,7th-circle,04:00,04:00,2022-05-16 10:42:32,2023-05-02 11:05:39,None
5,964fde3e-aec3-41f6-8fc9-b201a919e1dd,2,Events Backup (Previously Main Warehouse),events-backup-previously-main-warehouse,04:00,04:00,2022-05-16 10:42:32,2023-03-22 13:36:05,None
6,96ed24b8-517e-465f-b29c-ea84ec013766,2,Kia Showroom,kia-showroom,04:00,04:00,2022-08-02 15:04:34,2023-01-16 09:14:49,None
7,96fecd4d-69bd-4669-aeef-a4724c147e9b,2,Event Branch,event-branch,04:00,04:00,2022-08-11 09:45:02,2023-01-16 09:08:03,None
8,97c3bc91-eba9-4732-be73-ca9c80081518,2,Zarqa - Karama,zarqa-karama,04:00,04:00,2022-11-17 07:16:20,2023-01-16 09:13:56,None


# Products

In [24]:
last_page = call_foodics('products', 1, client_id,token, return_last_page=True)

list_responses = call_foodics('products', last_page, client_id,token, includables='category')

page 1
products?page=1
page 1
products?page=1&include=category
page 2
products?page=2&include=category
page 3
products?page=3&include=category
page 4
products?page=4&include=category
page 5
products?page=5&include=category
page 6
products?page=6&include=category
page 7
products?page=7&include=category
page 8
products?page=8&include=category


In [25]:
df_products = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_products['client_id'] = client_id
df_products['slug'] = df_products.name.apply(lambda x: generate_slug(x))

In [26]:
df_products.head()

df_products[df_products.id == '97164374-f596-4597-b128-dc5530e995ec']

,category,id,sku,barcode,name,name_localized,description,description_localized,image,is_active,is_stock_product,is_ready,pricing_method,selling_method,costing_method,preparation_time,price,cost,calories,created_at,updated_at,deleted_at,meta,client_id,slug
207,"{'id': '97161ba2-d907-4bb1-bb82-a22c9acbcb93',...",97164374-f596-4597-b128-dc5530e995ec,dine-pro34,None,Iced Hazelnut Latte,None,None,None,https://foodics-console-production.s3.eu-west-...,True,False,True,1,1,2,NaN,0.0,NaN,None,2022-08-23 01:39:28,2023-05-14 15:17:31,None,None,2,iced-hazelnut-latte


In [27]:
category_name_col = os.getenv('category_name_col')
print(category_name_col)

df_products['category_name'] = df_products['category'].apply(lambda x: x[category_name_col])
df_products = df_products[~df_products.category_name.isna()]
df_products['category_name'] = df_products['category_name'].apply(lambda x: x.capitalize())

df_products['category_id'] = df_products['category'].apply(lambda x: x['id'])
df_products.rename(columns={'sku':"sku"}, inplace=True)
df_products.drop('category', axis=1, inplace=True)
# df_products = df_products[df_products.deleted_at.isna()]
df_products.head()

name


,id,sku,barcode,name,name_localized,description,description_localized,image,is_active,is_stock_product,is_ready,pricing_method,selling_method,costing_method,preparation_time,price,cost,calories,created_at,updated_at,deleted_at,meta,client_id,slug,category_name,category_id
0,964fde65-52d5-491c-b227-da1603227e82,MER-013,None,Coffee Bean 250g (Delivery) - F,None,None,None,https://foodics-console-production.s3.eu-west-...,True,False,True,1,1,1,NaN,0.0,0.0,None,2022-05-16 10:42:57,2022-08-22 21:36:18,2022-08-22 21:36:18,None,2,coffee-bean-250g-delivery-f,Delivery food,96ecc1b0-0590-4e3d-9325-91b371c2a18a
1,964fde65-55c7-4cc3-84df-285efec97d70,MER-015,None,Matte Mug (Delivery) - F,None,None,None,https://foodics-console-production.s3.eu-west-...,True,False,True,1,1,1,NaN,0.0,0.0,None,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19,None,2,matte-mug-delivery-f,Delivery food,96ecc1b0-0590-4e3d-9325-91b371c2a18a
2,964fde65-57c6-450f-b537-38b8a3395ee0,sk-004,None,Hot chocolate-F (Hot chocolate-F),None,None,None,None,True,False,True,1,1,2,NaN,0.0,0.0,None,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19,None,2,hot-chocolate-f-hot-chocolate-f,Sweet coffee,964fde5b-9a21-44db-9dab-b98ff6a4f3d0
3,964fde65-5969-4b44-9a47-00849d643cb8,sk-007,None,Calibration (Calibration),None,None,None,None,True,False,True,1,1,2,NaN,0.0,0.0,None,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19,None,2,calibration-calibration,Other,964fde5b-55b6-4d27-af90-22bd41e2f54e
4,964fde65-5b14-4fee-a63d-757f5a46dce0,sk-005,None,Iced Lotus Latte-F (SUPER),None,None,None,https://foodics-console-production.s3.eu-west-...,True,False,True,1,1,2,NaN,0.0,NaN,None,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19,None,2,iced-lotus-latte-f-super,Sweet coffee,964fde5b-9a21-44db-9dab-b98ff6a4f3d0


In [28]:
df_products = df_products[['id', 'client_id', 'sku', 'slug', 'name', 'category_id', 'is_active', 'is_stock_product', 'price', 'created_at', 'updated_at', 'deleted_at']]

In [29]:

df_products = df_products.rename(columns={'is_stock_product':"is_stock"})

In [30]:
df_products.head()

,id,client_id,sku,slug,name,category_id,is_active,is_stock,price,created_at,updated_at,deleted_at
0,964fde65-52d5-491c-b227-da1603227e82,2,MER-013,coffee-bean-250g-delivery-f,Coffee Bean 250g (Delivery) - F,96ecc1b0-0590-4e3d-9325-91b371c2a18a,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:18,2022-08-22 21:36:18
1,964fde65-55c7-4cc3-84df-285efec97d70,2,MER-015,matte-mug-delivery-f,Matte Mug (Delivery) - F,96ecc1b0-0590-4e3d-9325-91b371c2a18a,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
2,964fde65-57c6-450f-b537-38b8a3395ee0,2,sk-004,hot-chocolate-f-hot-chocolate-f,Hot chocolate-F (Hot chocolate-F),964fde5b-9a21-44db-9dab-b98ff6a4f3d0,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
3,964fde65-5969-4b44-9a47-00849d643cb8,2,sk-007,calibration-calibration,Calibration (Calibration),964fde5b-55b6-4d27-af90-22bd41e2f54e,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
4,964fde65-5b14-4fee-a63d-757f5a46dce0,2,sk-005,iced-lotus-latte-f-super,Iced Lotus Latte-F (SUPER),964fde5b-9a21-44db-9dab-b98ff6a4f3d0,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19


# Categories

In [31]:
last_page = call_foodics('categories', 1, client_id,token, return_last_page=True)


list_responses = call_foodics('categories', last_page, client_id, token,) 

page 1
categories?page=1
page 1
categories?page=1


In [32]:
df_categories = pd.DataFrame([item for sublist in list_responses for item in sublist])
df_categories['client_id'] = client_id
df_categories['slug'] = df_categories.name.apply(lambda x: generate_slug(x))
df_categories.head()

,id,name,name_localized,reference,image,created_at,updated_at,deleted_at,client_id,slug
0,964fde5b-5595-4013-bd7f-312868cbfb0a,Milk Coffee,None,7,https://foodics-console-production.s3.eu-west-...,2022-05-16 10:42:51,2022-08-22 21:37:24,2022-08-22 21:37:24,2,milk-coffee
1,964fde5b-55b6-4d27-af90-22bd41e2f54e,Other,None,8,None,2022-05-16 10:42:51,2022-08-22 21:37:24,2022-08-22 21:37:24,2,other
2,964fde5b-685b-4324-ae2c-41bcf2382ebc,Offers,None,9,None,2022-05-16 10:42:51,2022-08-22 21:33:40,2022-08-22 21:33:40,2,offers
3,964fde5b-7c26-4c1e-9f85-4f17ec9fa7c6,Inactive,None,10,None,2022-05-16 10:42:51,2022-08-22 21:33:40,2022-08-22 21:33:40,2,inactive
4,964fde5b-9a21-44db-9dab-b98ff6a4f3d0,Sweet Coffee,None,11,https://foodics-console-production.s3.eu-west-...,2022-05-16 10:42:51,2022-08-22 21:37:24,2022-08-22 21:37:24,2,sweet-coffee


In [33]:
cats_to_be_deleted = df_categories[~df_categories.deleted_at.isna()]
cats_to_be_deleted.head()

,id,name,name_localized,reference,image,created_at,updated_at,deleted_at,client_id,slug
0,964fde5b-5595-4013-bd7f-312868cbfb0a,Milk Coffee,None,7,https://foodics-console-production.s3.eu-west-...,2022-05-16 10:42:51,2022-08-22 21:37:24,2022-08-22 21:37:24,2,milk-coffee
1,964fde5b-55b6-4d27-af90-22bd41e2f54e,Other,None,8,None,2022-05-16 10:42:51,2022-08-22 21:37:24,2022-08-22 21:37:24,2,other
2,964fde5b-685b-4324-ae2c-41bcf2382ebc,Offers,None,9,None,2022-05-16 10:42:51,2022-08-22 21:33:40,2022-08-22 21:33:40,2,offers
3,964fde5b-7c26-4c1e-9f85-4f17ec9fa7c6,Inactive,None,10,None,2022-05-16 10:42:51,2022-08-22 21:33:40,2022-08-22 21:33:40,2,inactive
4,964fde5b-9a21-44db-9dab-b98ff6a4f3d0,Sweet Coffee,None,11,https://foodics-console-production.s3.eu-west-...,2022-05-16 10:42:51,2022-08-22 21:37:24,2022-08-22 21:37:24,2,sweet-coffee


In [34]:
df_categories = df_categories[df_categories.deleted_at.isna()]

In [35]:
df_categories = df_categories[['id', 'client_id', category_name_col, 'slug', 'created_at', 'updated_at', 'deleted_at']].rename(columns={category_name_col:"name"})


In [36]:
df_categories

,id,client_id,name,slug,created_at,updated_at,deleted_at
16,97161ba2-d56a-4a37-a75f-1303a0c4a15c,2,Food,food,2022-08-22 23:48:07,2023-03-15 20:16:26,None
18,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,Sweet Coffee,sweet-coffee,2022-08-22 23:48:07,2023-03-15 20:16:37,None
19,97161ba2-da3c-4f23-8a52-218be3fb5fd7,2,Delivery Food,delivery-food,2022-08-22 23:48:07,2023-03-15 20:16:53,None
20,97161ba2-da47-449e-a2ff-c16deafb5c91,2,Milk Coffee,milk-coffee,2022-08-22 23:48:07,2022-08-22 23:48:07,None
22,97161ba2-dc03-4f8b-a35d-f1cc3d809906,2,Bottles,bottles,2022-08-22 23:48:07,2023-03-15 20:39:45,None
24,97161ba2-de90-4126-85dc-0d41ce1a266a,2,Black Coffee,black-coffee,2022-08-22 23:48:07,2023-03-15 20:35:35,None
25,97161ba2-df3b-47d3-a02e-8f950ae7033f,2,Delivery Drinks,delivery-drinks,2022-08-22 23:48:07,2022-08-22 23:48:07,None
26,97161ba2-e1ed-4711-8d43-3f0e71acc13c,2,Merchandise,merchandise,2022-08-22 23:48:07,2023-03-13 09:42:18,None
27,97161ba2-e228-4a6d-8d01-10e35ed3ec22,2,Other,other,2022-08-22 23:48:07,2022-08-22 23:48:07,None
28,98ad4709-821b-4bb6-8c8e-0157f7510d20,2,Delivery Merch,delivery-merch,2023-03-13 09:31:25,2023-03-15 20:39:36,None


In [37]:
df_categories.name = df_categories.name.apply(lambda x: x.capitalize())

In [38]:
print(df_categories['name'].tolist())

['Food', 'Sweet coffee', 'Delivery food', 'Milk coffee', 'Bottles', 'Black coffee', 'Delivery drinks', 'Merchandise', 'Other', 'Delivery merch', 'Lto (limited time offer)', 'Delivery lto (limited time offer)', 'Delivery black coffee', 'Delivery milk coffee', 'Delivery sweet coffee']


# Orders

In [39]:
filter = {}
path = f'../../data/{client_id}/raw/orders_final_include.csv'


if os.path.exists(path):
    print('orders_final_include.csv already exists')
    df_orders = pd.read_csv(path)
    df_orders.reset_index(drop=True, inplace=True)  
else:
    print('orders_final_include.csv does not exist')
    includables = 'branch,products.product,products.options.modifier_option'
    # call the foodics api to get the orders
    last_page = call_foodics('orders', last_page, client_id, token, includables=includables, filter=filter, return_last_page=True)
    print('last_page: ', last_page)

    #temp
    last_page = 5
    
    list_responses = call_foodics('orders', last_page, client_id, token, includables=includables, filter=filter)
    df_orders = pd.DataFrame([item for sublist in list_responses for item in sublist])
    df_orders.to_csv(path, index=False)

# TODO: remove this line after testing
df_orders = pd.read_csv(path)
df_orders.reset_index(drop=True, inplace=True)
    

orders_final_include.csv already exists


### Process Datetime columns

In [40]:
df_orders.created_at = pd.to_datetime(df_orders.created_at)
max_date = df_orders.created_at.max().date().strftime('%Y-%m-%d')
max_date

'2023-06-14'

In [41]:
today = date.today().strftime('%Y-%m-%d')
today

'2023-06-15'

In [42]:
df_orders.head()

,branch,products,id,app_id,promotion_id,discount_type,reference_x,number,type,source,status,delivery_status,guests,kitchen_notes,customer_notes,business_date,subtotal_price,discount_amount,rounding_amount,total_price,tax_exclusive_discount_amount,delay_in_seconds,meta,opened_at,accepted_at,due_at,driver_assigned_at,dispatched_at,driver_collected_at,delivered_at,closed_at,created_at,updated_at,reference,check_number
0,"{'id': '964fde3e-a16d-4136-b433-f81d2c1d9b12',...",[{'product': {'id': '97164374-f596-4597-b128-d...,0000220d-694e-466e-9e1e-763fba746540,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,4,2,1,4,1.0,1,NaN,NaN,2022-11-04,1.85185,0.0,0,2.00,0.0,NaN,{'foodics': {'device_id': '964fde3e-c5ae-4bef-...,2022-11-04 05:41:24,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-04 05:41:29,2022-11-04 05:43:37,2022-11-04 05:43:37,180671,150207
1,"{'id': '96fecd4d-69bd-4669-aeef-a4724c147e9b',...",[{'product': {'id': '97164376-e22a-4073-8096-3...,000035d9-e082-432c-934d-040985070e32,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,40,2,1,4,1.0,1,NaN,NaN,2023-01-10,0.86207,0.0,0,1.00,0.0,NaN,{'foodics': {'device_id': '96fee055-8cc0-4d31-...,2023-01-10 07:48:46,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-10 07:48:50,2023-01-10 07:49:43,2023-01-10 07:49:43,297125,106152
2,"{'id': '964fde3e-a16d-4136-b433-f81d2c1d9b12',...",[{'product': {'id': '97164374-f592-44e8-8089-2...,00006231-a6fd-4b53-a0d8-14ee300f190c,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,380,2,1,4,1.0,1,NaN,NaN,2022-11-15,1.85185,0.0,0,2.00,0.0,NaN,{'foodics': {'device_id': '964fde3e-c5ae-4bef-...,2022-11-15 19:33:38,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-15 19:33:41,2022-11-15 19:34:16,2022-11-15 19:34:16,199924,155353
3,"{'id': '964fde3e-abf8-476c-b139-cc314022c24f',...",[{'product': {'id': '964fde67-6f51-4870-8003-e...,000062f8-a0a7-4266-837a-080c1bc6d4eb,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,22,2,1,4,NaN,1,NaN,840666907.0,2022-08-05,3.23276,0.0,0,3.75,0.0,NaN,{'foodics': {'device_id': '964fde3e-bf84-4fc8-...,2022-08-05 08:14:01,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-05 08:14:29,2022-08-05 08:16:47,2022-08-05 08:16:47,8772,100405
4,"{'id': '97c3bc91-eba9-4732-be73-ca9c80081518',...",[{'product': {'id': '97164375-85f8-444e-b8f7-b...,0000969f-8a5b-4e6c-baa9-0928e1f84519,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,80,2,1,4,1.0,1,NaN,NaN,2023-04-22,1.85185,0.0,0,2.00,0.0,NaN,{'foodics': {'device_id': '97c3bce0-088b-42c0-...,2023-04-22 13:42:26,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-22 13:43:42,2023-04-22 13:44:04,2023-04-22 13:44:04,492810,130306


In [43]:
df_orders['branch'] = df_orders['branch'].astype(str)

In [44]:
df_orders['branch_id'] = df_orders['branch'].apply(lambda x: eval(x)['id'])
df_orders['client_id'] = client_id

In [45]:
orders_header = df_orders[['id', 'client_id', 'branch_id',  'source', 'type', 'status', 'total_price', 'created_at', 'updated_at']]
orders_header.isnull().sum()

id             0
client_id      0
branch_id      0
source         0
type           0
status         0
total_price    0
created_at     0
updated_at     0
dtype: int64

In [46]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at
0,0000220d-694e-466e-9e1e-763fba746540,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,1,2,4,2.00,2022-11-04 05:43:37,2022-11-04 05:43:37
1,000035d9-e082-432c-934d-040985070e32,2,96fecd4d-69bd-4669-aeef-a4724c147e9b,1,2,4,1.00,2023-01-10 07:49:43,2023-01-10 07:49:43
2,00006231-a6fd-4b53-a0d8-14ee300f190c,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,1,2,4,2.00,2022-11-15 19:34:16,2022-11-15 19:34:16
3,000062f8-a0a7-4266-837a-080c1bc6d4eb,2,964fde3e-abf8-476c-b139-cc314022c24f,1,2,4,3.75,2022-08-05 08:16:47,2022-08-05 08:16:47
4,0000969f-8a5b-4e6c-baa9-0928e1f84519,2,97c3bc91-eba9-4732-be73-ca9c80081518,1,2,4,2.00,2023-04-22 13:44:04,2023-04-22 13:44:04


In [47]:
orders_header['ordered_at'] = orders_header['created_at']

/var/folders/96/0p49z_3s7w5ft6_98hyb_nvc0000gq/T/ipykernel_68625/4153841500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_header['ordered_at'] = orders_header['created_at']


# Order Details

In [48]:
df_orders.head()

,branch,products,id,app_id,promotion_id,discount_type,reference_x,number,type,source,status,delivery_status,guests,kitchen_notes,customer_notes,business_date,subtotal_price,discount_amount,rounding_amount,total_price,tax_exclusive_discount_amount,delay_in_seconds,meta,opened_at,accepted_at,due_at,driver_assigned_at,dispatched_at,driver_collected_at,delivered_at,closed_at,created_at,updated_at,reference,check_number,branch_id,client_id
0,"{'id': '964fde3e-a16d-4136-b433-f81d2c1d9b12',...",[{'product': {'id': '97164374-f596-4597-b128-d...,0000220d-694e-466e-9e1e-763fba746540,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,4,2,1,4,1.0,1,NaN,NaN,2022-11-04,1.85185,0.0,0,2.00,0.0,NaN,{'foodics': {'device_id': '964fde3e-c5ae-4bef-...,2022-11-04 05:41:24,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-04 05:41:29,2022-11-04 05:43:37,2022-11-04 05:43:37,180671,150207,964fde3e-a16d-4136-b433-f81d2c1d9b12,2
1,"{'id': '96fecd4d-69bd-4669-aeef-a4724c147e9b',...",[{'product': {'id': '97164376-e22a-4073-8096-3...,000035d9-e082-432c-934d-040985070e32,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,40,2,1,4,1.0,1,NaN,NaN,2023-01-10,0.86207,0.0,0,1.00,0.0,NaN,{'foodics': {'device_id': '96fee055-8cc0-4d31-...,2023-01-10 07:48:46,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-10 07:48:50,2023-01-10 07:49:43,2023-01-10 07:49:43,297125,106152,96fecd4d-69bd-4669-aeef-a4724c147e9b,2
2,"{'id': '964fde3e-a16d-4136-b433-f81d2c1d9b12',...",[{'product': {'id': '97164374-f592-44e8-8089-2...,00006231-a6fd-4b53-a0d8-14ee300f190c,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,380,2,1,4,1.0,1,NaN,NaN,2022-11-15,1.85185,0.0,0,2.00,0.0,NaN,{'foodics': {'device_id': '964fde3e-c5ae-4bef-...,2022-11-15 19:33:38,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-15 19:33:41,2022-11-15 19:34:16,2022-11-15 19:34:16,199924,155353,964fde3e-a16d-4136-b433-f81d2c1d9b12,2
3,"{'id': '964fde3e-abf8-476c-b139-cc314022c24f',...",[{'product': {'id': '964fde67-6f51-4870-8003-e...,000062f8-a0a7-4266-837a-080c1bc6d4eb,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,22,2,1,4,NaN,1,NaN,840666907.0,2022-08-05,3.23276,0.0,0,3.75,0.0,NaN,{'foodics': {'device_id': '964fde3e-bf84-4fc8-...,2022-08-05 08:14:01,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-05 08:14:29,2022-08-05 08:16:47,2022-08-05 08:16:47,8772,100405,964fde3e-abf8-476c-b139-cc314022c24f,2
4,"{'id': '97c3bc91-eba9-4732-be73-ca9c80081518',...",[{'product': {'id': '97164375-85f8-444e-b8f7-b...,0000969f-8a5b-4e6c-baa9-0928e1f84519,8f7884c0-9e14-4833-ba96-3221471e8709,NaN,NaN,NaN,80,2,1,4,1.0,1,NaN,NaN,2023-04-22,1.85185,0.0,0,2.00,0.0,NaN,{'foodics': {'device_id': '97c3bce0-088b-42c0-...,2023-04-22 13:42:26,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-22 13:43:42,2023-04-22 13:44:04,2023-04-22 13:44:04,492810,130306,97c3bc91-eba9-4732-be73-ca9c80081518,2


In [67]:
# I want to create a dataframe with the following columns:
# order_id, product_id, category_id, quantity, price

# I will create a list of dictionaries, where each dictionary is a row in the dataframe
# I will then convert the list of dictionaries to a dataframe
import sys
import uuid

need_unavailable_category = False

list_order_details = []
length_orders = len(df_orders)
for index, row in df_orders.iterrows():

    order_header_id = row['id']
    branch_id = eval(row['branch'])['id']
    created_at = row['created_at']
    updated_at = row['updated_at']
    total_price = row['total_price']


    for order_product in eval(row['products']):
        order_details_id = str(uuid.uuid4())

        product_id = order_product['product']['id']
        try:
            category_id = df_products[df_products['id'] == product_id]['category_id'].values[0]
        except:
            need_unavailable_category = True
            category_id = '00000000-0000-0000-0000-000000000000'
        
        quantity = order_product['quantity']
        price = order_product['total_price']

        list_order_details.append({
                                   'id':order_details_id,
                                   'header_id':order_header_id,
                                   'product_id':product_id,
                                   'category_id':category_id,
                                   'client_id':client_id, 
                                   'quantity':quantity,
                                   'price':price,
                                   'created_at':created_at,
                                   'updated_at':updated_at})


In [68]:
if need_unavailable_category:
    print("Yes, we need to add the unavailable category, make sure to add it in the .env")
    new_row = pd.DataFrame({'id': '00000000-0000-0000-0000-000000000000', 'name': 'Not Available', 'slug':"not-available", 'client_id':client_id}, index=[df_categories.index.max()+1])
    today = date.today().strftime('%Y-%m-%d')
    df_categories = pd.concat([df_categories, new_row], ignore_index=True)
    df_categories.created_at = df_categories.created_at.fillna(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    df_categories.updated_at = df_categories.updated_at.fillna(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    df_categories.deleted_at = df_categories.deleted_at.fillna(method='ffill')



df_categories

,id,client_id,name,slug,created_at,updated_at,deleted_at
16,97161ba2-d56a-4a37-a75f-1303a0c4a15c,2,Food,food,2022-08-22 23:48:07,2023-03-15 20:16:26,None
18,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,Sweet coffee,sweet-coffee,2022-08-22 23:48:07,2023-03-15 20:16:37,None
19,97161ba2-da3c-4f23-8a52-218be3fb5fd7,2,Delivery food,delivery-food,2022-08-22 23:48:07,2023-03-15 20:16:53,None
20,97161ba2-da47-449e-a2ff-c16deafb5c91,2,Milk coffee,milk-coffee,2022-08-22 23:48:07,2022-08-22 23:48:07,None
22,97161ba2-dc03-4f8b-a35d-f1cc3d809906,2,Bottles,bottles,2022-08-22 23:48:07,2023-03-15 20:39:45,None
24,97161ba2-de90-4126-85dc-0d41ce1a266a,2,Black coffee,black-coffee,2022-08-22 23:48:07,2023-03-15 20:35:35,None
25,97161ba2-df3b-47d3-a02e-8f950ae7033f,2,Delivery drinks,delivery-drinks,2022-08-22 23:48:07,2022-08-22 23:48:07,None
26,97161ba2-e1ed-4711-8d43-3f0e71acc13c,2,Merchandise,merchandise,2022-08-22 23:48:07,2023-03-13 09:42:18,None
27,97161ba2-e228-4a6d-8d01-10e35ed3ec22,2,Other,other,2022-08-22 23:48:07,2022-08-22 23:48:07,None
28,98ad4709-821b-4bb6-8c8e-0157f7510d20,2,Delivery merch,delivery-merch,2023-03-13 09:31:25,2023-03-15 20:39:36,None


In [69]:
orders_details = pd.DataFrame(list_order_details)

In [70]:
orders_details.head()

,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at
0,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,0000220d-694e-466e-9e1e-763fba746540,97164374-f596-4597-b128-dc5530e995ec,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,1,2.00,2022-11-04 05:43:37,2022-11-04 05:43:37
1,bedfafdd-4194-41b4-a494-ad2264c10cc3,000035d9-e082-432c-934d-040985070e32,97164376-e22a-4073-8096-3e40dbd5c1b7,97161ba2-de90-4126-85dc-0d41ce1a266a,2,1,1.00,2023-01-10 07:49:43,2023-01-10 07:49:43
2,c35b357d-414f-430d-a032-79a111046428,00006231-a6fd-4b53-a0d8-14ee300f190c,97164374-f592-44e8-8089-242fef28e599,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,1,2.00,2022-11-15 19:34:16,2022-11-15 19:34:16
3,0ea512d3-15f5-4f6c-acc6-d273bd1a08d8,000062f8-a0a7-4266-837a-080c1bc6d4eb,964fde67-6f51-4870-8003-e1cfaa0b3ccb,964fde5c-05f2-4ed5-a55e-882453ad5311,2,1,2.50,2022-08-05 08:16:47,2022-08-05 08:16:47
4,1e4e0368-5124-443f-8479-d7fa1dd4954f,000062f8-a0a7-4266-837a-080c1bc6d4eb,964fde6f-82e4-43e8-96a7-d39cd5a922d4,964fde5c-05f2-4ed5-a55e-882453ad5311,2,1,1.25,2022-08-05 08:16:47,2022-08-05 08:16:47


In [71]:
orders_details[orders_details.category_id.isna()].sort_values('created_at')

,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at


# Order Details Options

In [72]:
df_options = pd.DataFrame(eval(df_orders.head()['products'][0])[0]['options'])
df_options.head()

#iterate over the df_orders dataframe and then iterate over the products column and then iterate over the options column
# and then iterate over the modifier_option column

list_order_options = []
for index, row in df_orders.iterrows():
    order_header_id = row['id']
    for order_product in eval(row['products']):
        product_id = order_product['product']['id']
        for order_option in order_product['options']:
            modifer_option = order_option['modifier_option']
            #get from modifier_option
            option_name = modifer_option['name']
            option_name_localized = modifer_option['name_localized']
            option_sku = modifer_option['sku']

            option_id = order_option['id']
            option_quantity = order_option['quantity']
            option_partition = order_option['partition']
            option_unit_price = order_option['unit_price']
            option_total_price = order_option['total_price']
            option_total_cost = order_option['total_cost']

            # get order_details_id by filtering df_order_details on product_id and order_header_id
            order_details_id = orders_details[(orders_details['product_id'] == product_id) & (orders_details['header_id'] == order_header_id)]['id'].values[0]

            list_order_options.append({
                'order_details_id': order_details_id,
                'option_id': option_id,
                'option_name': option_name,
                'option_name_localized': option_name_localized,
                'option_sku': option_sku,
                'option_quantity': option_quantity,
                'option_partition': option_partition,
                'option_unit_price': option_unit_price,
                'option_total_price': option_total_price,
                'option_total_cost': option_total_cost
            })
                                        
df_options = pd.DataFrame(list_order_options)

In [73]:
# filter df_options on order_details_id and option_id
df_options.head()

,order_details_id,option_id,option_name,option_name_localized,option_sku,option_quantity,option_partition,option_unit_price,option_total_price,option_total_cost
0,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,97a9749c-e3c1-4ff6-b190-690a3a0c1814,Regular,عادي,option-modifier-9924,1,1,2.0,2.0,0.43524
1,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,97a9749c-e58b-404b-9676-7aa86f54f967,Full Fat,حليب كامل الدسم,option-modifier-9882,1,1,0.0,0.0,0.05333
2,bedfafdd-4194-41b4-a494-ad2264c10cc3,983069a5-e226-4ec9-97f3-a78be49d0ff4,Regular,عادي,option-modifier-9868,1,1,1.0,1.0,0.19303
3,c35b357d-414f-430d-a032-79a111046428,97c0be81-4a84-4192-8adf-39944db65dcd,Regular,عادي,option-modifier-9933,1,1,2.0,2.0,0.49796
4,c35b357d-414f-430d-a032-79a111046428,97c0be81-4ceb-48b4-8872-1a01799e0e31,Full Fat,حليب كامل الدسم,option-modifier-9882,1,1,0.0,0.0,0.05333


In [74]:
#grab an id from order_details
order_details_id = orders_details['id'].values[0]

df_options[df_options['order_details_id'] == order_details_id]

orders_details[orders_details['id'] == order_details_id]

,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at
0,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,0000220d-694e-466e-9e1e-763fba746540,97164374-f596-4597-b128-dc5530e995ec,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,1,2.0,2022-11-04 05:43:37,2022-11-04 05:43:37


In [66]:
# There is no need to recalculate the order_details price by summing product price and options price. The products array comes with total price already calculated for each order details.

## KEEP THIS CODE FOR REFERENCE IF WE WANT TO MANUALLY CALCULATE THE ORDER_DETAILS PRICE##
# we need to adjust all order_details price coloumn, to be the sum of the column value and the sum of the df_options total_price column
# loop over the df_options dataframe and for each order_details_id, sum the total_price column and then update the orders_details dataframe

# for index, row in df_options.iterrows():
#     order_details_id = row['order_details_id']
#     total_price = row['option_total_price']
#     orders_details.loc[orders_details['id'] == order_details_id, 'price'] += total_price
    
# orders_details[orders_details['id'] == order_details_id]

,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at
384,73c43f11-a035-461a-b957-7ad8e713963b,001b3f87-6a73-4153-80fd-34f8039e9924,97164375-066d-414c-ba55-c778f770b504,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,1,4.0,2022-08-25 08:02:03,2022-08-25 08:02:03


## Change Data Timezone from UTC to Asia/Amm

In [75]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,0000220d-694e-466e-9e1e-763fba746540,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,1,2,4,2.00,2022-11-04 05:43:37,2022-11-04 05:43:37,2022-11-04 05:43:37
1,000035d9-e082-432c-934d-040985070e32,2,96fecd4d-69bd-4669-aeef-a4724c147e9b,1,2,4,1.00,2023-01-10 07:49:43,2023-01-10 07:49:43,2023-01-10 07:49:43
2,00006231-a6fd-4b53-a0d8-14ee300f190c,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,1,2,4,2.00,2022-11-15 19:34:16,2022-11-15 19:34:16,2022-11-15 19:34:16
3,000062f8-a0a7-4266-837a-080c1bc6d4eb,2,964fde3e-abf8-476c-b139-cc314022c24f,1,2,4,3.75,2022-08-05 08:16:47,2022-08-05 08:16:47,2022-08-05 08:16:47
4,0000969f-8a5b-4e6c-baa9-0928e1f84519,2,97c3bc91-eba9-4732-be73-ca9c80081518,1,2,4,2.00,2023-04-22 13:44:04,2023-04-22 13:44:04,2023-04-22 13:44:04


In [76]:
orders_details.head()

,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at
0,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,0000220d-694e-466e-9e1e-763fba746540,97164374-f596-4597-b128-dc5530e995ec,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,1,2.00,2022-11-04 05:43:37,2022-11-04 05:43:37
1,bedfafdd-4194-41b4-a494-ad2264c10cc3,000035d9-e082-432c-934d-040985070e32,97164376-e22a-4073-8096-3e40dbd5c1b7,97161ba2-de90-4126-85dc-0d41ce1a266a,2,1,1.00,2023-01-10 07:49:43,2023-01-10 07:49:43
2,c35b357d-414f-430d-a032-79a111046428,00006231-a6fd-4b53-a0d8-14ee300f190c,97164374-f592-44e8-8089-242fef28e599,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,1,2.00,2022-11-15 19:34:16,2022-11-15 19:34:16
3,0ea512d3-15f5-4f6c-acc6-d273bd1a08d8,000062f8-a0a7-4266-837a-080c1bc6d4eb,964fde67-6f51-4870-8003-e1cfaa0b3ccb,964fde5c-05f2-4ed5-a55e-882453ad5311,2,1,2.50,2022-08-05 08:16:47,2022-08-05 08:16:47
4,1e4e0368-5124-443f-8479-d7fa1dd4954f,000062f8-a0a7-4266-837a-080c1bc6d4eb,964fde6f-82e4-43e8-96a7-d39cd5a922d4,964fde5c-05f2-4ed5-a55e-882453ad5311,2,1,1.25,2022-08-05 08:16:47,2022-08-05 08:16:47


In [77]:
# import pytz
from datetime import timedelta

# orders_details['ordered_at'] = pd.to_datetime(orders_details['order_date'])
# # orders_details['order_date'] = orders_details['order_date'].dt.tz_localize('UTC')
# # orders_details['order_date'] = orders_details['order_date'].dt.tz_convert(pytz.timezone('Asia/Amman'))

orders_header['ordered_at'] = pd.to_datetime(orders_header['ordered_at'])
# orders_header['order_date'] = orders_header['order_date'].dt.tz_localize('UTC')
# orders_header['order_date'] = orders_header['order_date'].dt.tz_convert(pytz.timezone('Asia/Amman'))

time_difference = timedelta(hours=business_timezone_offset)

orders_header['ordered_at'] += time_difference
# orders_details['order_date'] += time_difference

/var/folders/96/0p49z_3s7w5ft6_98hyb_nvc0000gq/T/ipykernel_68625/1726249347.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_header['ordered_at'] = pd.to_datetime(orders_header['ordered_at'])
/var/folders/96/0p49z_3s7w5ft6_98hyb_nvc0000gq/T/ipykernel_68625/1726249347.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_header['ordered_at'] += time_difference


In [78]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,0000220d-694e-466e-9e1e-763fba746540,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,1,2,4,2.00,2022-11-04 05:43:37,2022-11-04 05:43:37,2022-11-04 08:43:37
1,000035d9-e082-432c-934d-040985070e32,2,96fecd4d-69bd-4669-aeef-a4724c147e9b,1,2,4,1.00,2023-01-10 07:49:43,2023-01-10 07:49:43,2023-01-10 10:49:43
2,00006231-a6fd-4b53-a0d8-14ee300f190c,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,1,2,4,2.00,2022-11-15 19:34:16,2022-11-15 19:34:16,2022-11-15 22:34:16
3,000062f8-a0a7-4266-837a-080c1bc6d4eb,2,964fde3e-abf8-476c-b139-cc314022c24f,1,2,4,3.75,2022-08-05 08:16:47,2022-08-05 08:16:47,2022-08-05 11:16:47
4,0000969f-8a5b-4e6c-baa9-0928e1f84519,2,97c3bc91-eba9-4732-be73-ca9c80081518,1,2,4,2.00,2023-04-22 13:44:04,2023-04-22 13:44:04,2023-04-22 16:44:04


# Write to CSV

In [79]:
df_products.to_csv(f'../../data/{client_id}/raw/products.csv', index=False)

In [80]:
orders_details.to_csv(f'../../data/{client_id}/raw/order_details.csv', index=False)

In [81]:
orders_header.to_csv(f'../../data/{client_id}/raw/order_header.csv', index=False)

In [82]:
df_categories.to_csv(f'../../data/{client_id}/raw/categories.csv', index=False)

In [83]:
df_branches.to_csv(f'../../data/{client_id}/raw/branches.csv', index=False)

In [84]:
df_options.to_csv(f'../../data/{client_id}/raw/options.csv', index=False)

# Display all

In [85]:
df_products.head()

,id,client_id,sku,slug,name,category_id,is_active,is_stock,price,created_at,updated_at,deleted_at
0,964fde65-52d5-491c-b227-da1603227e82,2,MER-013,coffee-bean-250g-delivery-f,Coffee Bean 250g (Delivery) - F,96ecc1b0-0590-4e3d-9325-91b371c2a18a,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:18,2022-08-22 21:36:18
1,964fde65-55c7-4cc3-84df-285efec97d70,2,MER-015,matte-mug-delivery-f,Matte Mug (Delivery) - F,96ecc1b0-0590-4e3d-9325-91b371c2a18a,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
2,964fde65-57c6-450f-b537-38b8a3395ee0,2,sk-004,hot-chocolate-f-hot-chocolate-f,Hot chocolate-F (Hot chocolate-F),964fde5b-9a21-44db-9dab-b98ff6a4f3d0,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
3,964fde65-5969-4b44-9a47-00849d643cb8,2,sk-007,calibration-calibration,Calibration (Calibration),964fde5b-55b6-4d27-af90-22bd41e2f54e,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19
4,964fde65-5b14-4fee-a63d-757f5a46dce0,2,sk-005,iced-lotus-latte-f-super,Iced Lotus Latte-F (SUPER),964fde5b-9a21-44db-9dab-b98ff6a4f3d0,True,False,0.0,2022-05-16 10:42:57,2022-08-22 21:36:19,2022-08-22 21:36:19


In [86]:
orders_details.head()

,id,header_id,product_id,category_id,client_id,quantity,price,created_at,updated_at
0,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,0000220d-694e-466e-9e1e-763fba746540,97164374-f596-4597-b128-dc5530e995ec,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,1,2.00,2022-11-04 05:43:37,2022-11-04 05:43:37
1,bedfafdd-4194-41b4-a494-ad2264c10cc3,000035d9-e082-432c-934d-040985070e32,97164376-e22a-4073-8096-3e40dbd5c1b7,97161ba2-de90-4126-85dc-0d41ce1a266a,2,1,1.00,2023-01-10 07:49:43,2023-01-10 07:49:43
2,c35b357d-414f-430d-a032-79a111046428,00006231-a6fd-4b53-a0d8-14ee300f190c,97164374-f592-44e8-8089-242fef28e599,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,1,2.00,2022-11-15 19:34:16,2022-11-15 19:34:16
3,0ea512d3-15f5-4f6c-acc6-d273bd1a08d8,000062f8-a0a7-4266-837a-080c1bc6d4eb,964fde67-6f51-4870-8003-e1cfaa0b3ccb,964fde5c-05f2-4ed5-a55e-882453ad5311,2,1,2.50,2022-08-05 08:16:47,2022-08-05 08:16:47
4,1e4e0368-5124-443f-8479-d7fa1dd4954f,000062f8-a0a7-4266-837a-080c1bc6d4eb,964fde6f-82e4-43e8-96a7-d39cd5a922d4,964fde5c-05f2-4ed5-a55e-882453ad5311,2,1,1.25,2022-08-05 08:16:47,2022-08-05 08:16:47


In [87]:
orders_header.head()

,id,client_id,branch_id,source,type,status,total_price,created_at,updated_at,ordered_at
0,0000220d-694e-466e-9e1e-763fba746540,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,1,2,4,2.00,2022-11-04 05:43:37,2022-11-04 05:43:37,2022-11-04 08:43:37
1,000035d9-e082-432c-934d-040985070e32,2,96fecd4d-69bd-4669-aeef-a4724c147e9b,1,2,4,1.00,2023-01-10 07:49:43,2023-01-10 07:49:43,2023-01-10 10:49:43
2,00006231-a6fd-4b53-a0d8-14ee300f190c,2,964fde3e-a16d-4136-b433-f81d2c1d9b12,1,2,4,2.00,2022-11-15 19:34:16,2022-11-15 19:34:16,2022-11-15 22:34:16
3,000062f8-a0a7-4266-837a-080c1bc6d4eb,2,964fde3e-abf8-476c-b139-cc314022c24f,1,2,4,3.75,2022-08-05 08:16:47,2022-08-05 08:16:47,2022-08-05 11:16:47
4,0000969f-8a5b-4e6c-baa9-0928e1f84519,2,97c3bc91-eba9-4732-be73-ca9c80081518,1,2,4,2.00,2023-04-22 13:44:04,2023-04-22 13:44:04,2023-04-22 16:44:04


In [88]:
df_branches.head()

,id,client_id,name,slug,opening_from,opening_to,created_at,updated_at,deleted_at
0,964fde3e-a16d-4136-b433-f81d2c1d9b12,2,Abdali Boulevard,abdali-boulevard,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:15:16,None
1,964fde3e-a371-426e-a209-d6f3ff79d2a1,2,Sports City,sports-city,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:16:20,None
2,964fde3e-a88f-4426-b476-62d018d33feb,2,Abdoun,abdoun,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:17:01,None
3,964fde3e-aa4d-48aa-8c29-e0c8c77cf0b6,2,Fuhais,fuhais,04:00,04:00,2022-05-16 10:42:32,2023-01-16 09:17:23,None
4,964fde3e-abf8-476c-b139-cc314022c24f,2,7th Circle,7th-circle,04:00,04:00,2022-05-16 10:42:32,2023-05-02 11:05:39,None


In [89]:
df_categories.head()

,id,client_id,name,slug,created_at,updated_at,deleted_at
16,97161ba2-d56a-4a37-a75f-1303a0c4a15c,2,Food,food,2022-08-22 23:48:07,2023-03-15 20:16:26,None
18,97161ba2-d907-4bb1-bb82-a22c9acbcb93,2,Sweet coffee,sweet-coffee,2022-08-22 23:48:07,2023-03-15 20:16:37,None
19,97161ba2-da3c-4f23-8a52-218be3fb5fd7,2,Delivery food,delivery-food,2022-08-22 23:48:07,2023-03-15 20:16:53,None
20,97161ba2-da47-449e-a2ff-c16deafb5c91,2,Milk coffee,milk-coffee,2022-08-22 23:48:07,2022-08-22 23:48:07,None
22,97161ba2-dc03-4f8b-a35d-f1cc3d809906,2,Bottles,bottles,2022-08-22 23:48:07,2023-03-15 20:39:45,None


In [90]:
df_options.head()

,order_details_id,option_id,option_name,option_name_localized,option_sku,option_quantity,option_partition,option_unit_price,option_total_price,option_total_cost
0,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,97a9749c-e3c1-4ff6-b190-690a3a0c1814,Regular,عادي,option-modifier-9924,1,1,2.0,2.0,0.43524
1,59d8a996-e13c-494d-9c8d-9b6fdfc0f230,97a9749c-e58b-404b-9676-7aa86f54f967,Full Fat,حليب كامل الدسم,option-modifier-9882,1,1,0.0,0.0,0.05333
2,bedfafdd-4194-41b4-a494-ad2264c10cc3,983069a5-e226-4ec9-97f3-a78be49d0ff4,Regular,عادي,option-modifier-9868,1,1,1.0,1.0,0.19303
3,c35b357d-414f-430d-a032-79a111046428,97c0be81-4a84-4192-8adf-39944db65dcd,Regular,عادي,option-modifier-9933,1,1,2.0,2.0,0.49796
4,c35b357d-414f-430d-a032-79a111046428,97c0be81-4ceb-48b4-8872-1a01799e0e31,Full Fat,حليب كامل الدسم,option-modifier-9882,1,1,0.0,0.0,0.05333
